In [1]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:2242/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

def get_llama_response(prompt, temperature = 0.1, max_tokens = 128):
    completion = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature = temperature,
        # max_tokens = max_tokens
    )

    return completion.choices[0].message.content

In [2]:
get_llama_response("৫ জন বাংলাদেশি বিখ্যাত কবির নাম বল?")

'আমার কাছে ৫ জন বাংলাদেশি প্রসিদ্ধ কবির নাম রয়েছে, যারা বাংলা সাহিত্যে অবদান রেখেছেন। তারা হলেন:\n\n১. রফিকউজ্জমান (রাফি)\n২. কাজী নজরুল ইসলাম\n৩. কাজী নজীর হোসেন\n৪. শামসুর রাশিদ চৌধুরী\n৫. শীল দাশগুপ্ত'

In [3]:
zero_shot = """আপনাকে একটি প্রশ্ন এবং চারটি বিকল্প উত্তর দেওয়া হয়েছে। আপনাকে সঠিক উত্তরটি নির্ধারণ করতে হবে এবং শুধুমাত্র ক, খ, গ, ঘ বিকল্পের মধ্যে থেকে সঠিক উত্তরটি লিখতে হবে।

উত্তরের জন্য প্রথমে অপশন নাম্বারটি লিখে (ক / খ/ গ / ঘ) ব্রাকেট ) দিয়ে উত্তর টি লিখুন। অপশনের মতোই নিম্নোক্ত ফরম্যাটটি অনুসরণ করুন।
ক / খ / গ / ঘ) উত্তর

"""

In [4]:
three_shot = """আপনাকে একটি প্রশ্ন এবং চারটি বিকল্প উত্তর দেওয়া হয়েছে। আপনাকে সঠিক উত্তরটি নির্ধারণ করতে হবে এবং শুধুমাত্র ক, খ, গ, ঘ বিকল্পের মধ্যে থেকে সঠিক উত্তরটি লিখতে হবে।

উত্তরের জন্য প্রথমে অপশন নাম্বারটি লিখে (ক / খ/ গ / ঘ) ব্রাকেট ) দিয়ে উত্তর টি লিখুন। অপশনের মতোই নিম্নোক্ত ফরম্যাটটি অনুসরণ করুন।
ক / খ / গ / ঘ) উত্তর

### উদাহরণ ১
### প্রশ্ন:
p^{2} + q^{2} = 3, pq = 3 হলে (p + q)^{2} এর মান কত?

### বিকল্প:
ক) 3
খ) 9
গ) 11
ঘ) 15

### আপনার উত্তর:
খ) 9

### উদাহরণ ২
### প্রশ্ন:
x - 3y = 15 এবং 3x + y = 3ন রেখাদ্বয়ের ঢালের গুণফল কত?

### বিকল্প:
ক) -3
খ) -1
গ) 1
ঘ) 3

### আপনার উত্তর:
খ) -1

### উদাহরণ ৩
### প্রশ্ন:
বিনা বাধায় পড়ন্ত বস্তু 5 সেকেন্ডে 50 মিটার পথ অতিক্রম করলে 72 মিটার পথ অতিক্রম করতে কত সেকেন্ড সময় লাগবে?

### বিকল্প:
ক) 6
খ) 7.2
গ) 9.5
ঘ) 12

### আপনার উত্তর:
ক) 6

### আপনার কাজ

"""

In [5]:
chain_of_thought = """আপনাকে একটি প্রশ্ন এবং চারটি বিকল্প উত্তর দেওয়া হয়েছে। আপনাকে সঠিক উত্তরটি নির্ধারণ করতে হবে এবং শুধুমাত্র ক, খ, গ, ঘ বিকল্পের মধ্যে থেকে সঠিক উত্তরটি লিখতে হবে।

উত্তরের জন্য প্রথমে অপশন নাম্বার টি লিখে (ক / খ/ গ / ঘ) ব্রাকেট ) দিয়ে উত্তর টি লিখুন। অপশনের মতোই নিম্নোক্ত ফরম্যাটটি অনুসরণ করুন।
ক / খ / গ / ঘ) উত্তর

প্রশ্নটির উত্তর নির্ধারণ করতে গিয়ে নিম্নলিখিত ধাপগুলি অনুসরণ করুন:

### ধাপ ১: প্রশ্নটি বোঝা
প্রশ্নটি ভাল করে পড়ুন এবং এটি কী জানতে চাচ্ছে তা বোঝার চেষ্টা করুন। প্রশ্নটির প্রয়োজনীয়তা এবং কনটেক্সটটি বোঝার জন্য এটি বিস্তারিতভাবে বিবেচনা করুন।

### ধাপ ২: বিকল্পগুলি পড়া
প্রতিটি বিকল্প ভাল করে পড়ুন এবং কোন কোন বিকল্পগুলি সম্ভবত সঠিক হতে পারে তা সনাক্ত করুন। প্রতিটি বিকল্পের সাথে প্রশ্নটির সম্পর্ক বিচার করুন।

### ধাপ ৩: বিকল্পগুলি মূল্যায়ন
প্রশ্নটির সাথে সম্পর্কিত জ্ঞান এবং তথ্য ব্যবহার করে সম্ভাব্য সমাধানে পৌছান। কোন বিকল্পটি সঠিক হতে পারে তা বিচার করুন।

### ধাপ ৪: সঠিক উত্তর নির্বাচন
প্রথম তিনটি ধাপের ভিত্তিতে সবচেয়ে যুক্তিযুক্ত এবং সঠিক উত্তরটি নির্বাচন করুন।

### উদাহরণ ১
### প্রশ্ন:
p^{2} + q^{2} = 3, pq = 3 হলে (p + q)^{2} এর মান কত?

### বিকল্প:
ক) 3
খ) 9
গ) 11
ঘ) 15

#### চিন্তাধারা:
- ধাপ ১: প্রশ্নটি বোঝা: এখানে, p^{2} + q^{2} এবং pq এর মান দেওয়া হয়েছে এবং (p + q)^{2} এর মান নির্ধারণ করতে বলা হয়েছে।
- ধাপ ২: বিকল্পগুলি পড়া: বিকল্পগুলি হল ৩, ৯, ১১, এবং ১৫।
- ধাপ ৩: সম্ভাব্য সমাধান: আমরা জানি যে (p + q)^{2} = p^{2} + 2pq + q^{2}। সুতরাং, (p + q)^{2} = 3 + 2*3 = 9।
- ধাপ ৪: সঠিক উত্তর নির্বাচন: সুতরাং, সঠিক উত্তর হল খ) ৯।

### আপনার উত্তর:
খ) ৯

### উদাহরণ ২
### প্রশ্ন:
x - 3y = 15 এবং 3x + y = 3ন রেখাদ্বয়ের ঢালের গুণফল কত?

### বিকল্প:
ক) -3
খ) -1
গ) 1
ঘ) 3

#### চিন্তাধারা:
- ধাপ ১: প্রশ্নটি বোঝা: এখানে, দুইটি রেখার সমীকরণ দেওয়া হয়েছে এবং তাদের ঢালের গুণফল নির্ধারণ করতে বলা হয়েছে।
- ধাপ ২: বিকল্পগুলি পড়া: বিকল্পগুলি হল -৩, -১, ১, এবং ৩।
- ধাপ ৩: সম্ভাব্য সমাধান: আমরা x - 3y = 15 কে y = mx + c আকারে লিখতে পারি: y = 1/3x + 15। ফলে, প্রথম রেখার ঢাল হল ১/৩ এবং দ্বিতীয় রেখার ঢাল হল -৩। সুতরাং, ঢালের গুণফল হল (১/৩) * (-৩) = -১।
- ধাপ ৪: সঠিক উত্তর নির্বাচন: সুতরাং, সঠিক উত্তর হল খ) -১।

### আপনার উত্তর:
খ) -১

### উদাহরণ ৩
### প্রশ্ন:
বিনা বাধায় পড়ন্ত বস্তু 5 সেকেন্ডে 50 মিটার পথ অতিক্রম করলে 72 মিটার পথ অতিক্রম করতে কত সেকেন্ড সময় লাগবে?

### বিকল্প:
ক) 6
খ) 7.2
গ) 9.5
ঘ) 12

#### চিন্তাধারা:
- ধাপ ১: প্রশ্নটি বোঝা: এখানে, বিনা বাধায় পড়ন্ত বস্তুর সময় নির্ধারণ করা।
- ধাপ ২: বিকল্পগুলি পড়া: বিকল্পগুলি হল ৬, ৭.২, ৯.৫, এবং ১২।
- ধাপ ৩: সম্ভাব্য সমাধান: বিনা বাধায় পড়ন্ত বস্তু t সময়ে h দূরত্ব অতিক্রম করে, যেখানে h = 1/2 * g * t^2। 50 মিটার যেতে 5 সেকেন্ড সময় লাগে: 50 = 1/2 * g * 5^2, g = 4 মিটার/সেকেন্ড^2। সুতরাং, 72 মিটার যেতে সময় লাগবে: 72 = 1/2 * 4 * t^2, t^2 = 36, t = 6 সেকেন্ড।
- ধাপ ৪: সঠিক উত্তর নির্বাচন: সুতরাং, সঠিক উত্তর হল ক) ৬।

### আপনার উত্তর:
খ) ৭.২

### আপনার কাজ


"""

In [6]:
def prepare_questions(row, base_prompt, prompt_type = 'zero_shot'):
    question = row['Bengali Question']
    options = {
        'ক': row['A Bn'],
        'খ': row['B Bn'],
        'গ': row['C Bn'],
        'ঘ': row['D Bn']
    }
    correct_answer = row['Correct Answer']
    eng2ben_map = {
        'a': 'ক',
        'b': 'খ',
        'c': 'গ',
        'd': 'ঘ'
    }
    correct_option = eng2ben_map[correct_answer] + ") " + options[eng2ben_map[correct_answer]]

    options_string = "\n".join([f"{key}) {value}" for key, value in options.items()])

    if prompt_type == 'zero_shot':
        question_prompt = base_prompt + f"### প্রশ্ন:\n{question}\n\n### বিকল্প:\n{options_string}\n\n### আপনার উত্তর:\n"
    elif prompt_type == 'three_shot':
        question_prompt = base_prompt + f"### প্রশ্ন:\n{question}\n\n### বিকল্প:\n{options_string}\n\n### আপনার উত্তর:\n"
    elif prompt_type == 'cot':
        question_prompt = base_prompt + f"### প্রশ্ন:\n{question}\n\n### বিকল্প:\n{options_string}\n\n"

    return question_prompt, correct_option

In [7]:
file_paths = [
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/8th-Science.csv",
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/12th-Math-I.csv",
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/12th-Math-II.csv",
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/12th-Physics-I.csv",
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/12th-Physics-II.csv"
]

file_tags = [
    "8th_science",
    "12th_math_1",
    "12th_math_2",
    "12th_physics_1",
    "12th_physics_2"
]

In [8]:
path_root = 'exps/'

In [14]:
import pandas as pd
df_progress = pd.read_parquet(path_root + f"llama_zs_rqa_8th_science_results.parquet")

In [15]:
len(df_progress)

228

In [16]:
df_progress.head()

,English Question,Bengali Question,A,B,C,D,A Bn,B Bn,C Bn,D Bn,Correct Answer,predicted_option,correct_option_fmt
0,Which acid present in tamarind?,তেঁতুলে কোন এসিড থাকে?,Malic,Citric,Oxalic,Tartaric,ম্যালিক,সাইট্রিক,অক্সালিক,টারটারিক,d,(খ) সাইট্রিক,ঘ) টারটারিক
1,By which two elements fuse is made?,কোন মৌল দুটির সাহায্যে ফিউজ প্রস্তুত করা হয়?,Lead and Zinc,Zinc and Tin,Lead and Tin,Lead and Iron,সীসা ও জিঙ্ক,জিঙ্জ ও টিন,সীসা ও টিন,সীসা ও লোহা,c,ক / ঘ) সীসা ও লোহা,গ) সীসা ও টিন
2,Which on is the tertiary consumer of sundarbans?,সুন্দরবনের তৃতীয় স্তরের খাদক কোনটি?,Pig,Deer,Monkey,Hen,শুকর,হরিণ,বানর,মুরগি,a,(গ) বানর\n\nবানর হল সুন্দরবনের একটি উল্লেখযোগ্...,ক) শুকর
3,Which hormone is sybtgesuzed in leaves?,পাতায় উৎপন্ন হরমোন কোনটি?,Auxin,Ethylene,Florigen,Gibberellin,অক্সিন,ইথিলিন,ফ্লোরিজেন,জিব্বেরেলিন,c,(খ) ইথিলিন,গ) ফ্লোরিজেন
4,Which one is used as insecticide?,ফসলের ক্ষতিকর পেস্ট দমন করে কোনটি?,NH_{4}OH,NaOH,Mg(OH)_{2},Ca(OH)_{2},NH_{4}OH,NaOH,Mg(OH)_{2},Ca(OH)_{2},d,(ঘ) Ca(OH)_{2},ঘ) Ca(OH)_{2}


In [17]:
import time
import pandas as pd
from tqdm.notebook import tqdm


def prepare_zs_results(df, dataset_tag):
    # Assuming df is the original DataFrame and category_translation is defined
    if 'predicted_option' not in df:
        df['predicted_option'] = None
    if 'correct_option_fmt' not in df:
        df['correct_option_fmt'] = None

    # Try to load the progress from the Parquet file
    try:
        df_progress = pd.read_parquet(path_root + f"llama_zs_rqa_{dataset_tag}_results.parquet")
        if df_progress['predicted_option'].last_valid_index() == None:
            start_index = 0
        else:
            start_index = df_progress['predicted_option'].last_valid_index() + 1
            start_index = 0
        df = df_progress.copy()
    except (FileNotFoundError, ValueError):
        df_progress = df.copy()
        start_index = 0


    for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):

        try:
            prompt_input, cor_opt = prepare_questions(row, base_prompt = zero_shot, prompt_type = 'zero_shot')
        except:
            print("Error w/ sample", row)

        if df.at[i, 'predicted_option'] != None:
            continue
            
        try:
            df.at[i, 'predicted_option'] = get_llama_response(prompt_input)
            df.at[i, 'correct_option_fmt'] = cor_opt
        except Exception as e:
            print(f"Error at index {i}")
            time.sleep(5)

        if (i + 1) % 25 == 0:
            df.to_parquet(path_root + f"llama_zs_rqa_{dataset_tag}_results.parquet")

    # Save final progress
    df.to_parquet(path_root + f"llama_zs_rqa_{dataset_tag}_results.parquet")
    return df

### set file index

for file_index in range(5):
    print(file_paths[file_index])
    df = pd.read_csv(file_paths[file_index])

    print(df.head())

    zs_results = prepare_zs_results(df, dataset_tag = file_tags[file_index])
    print(zs_results.head())

https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/8th-Science.csv
                                   English Question  \
0                   Which acid present in tamarind?   
1               By which two elements fuse is made?   
2  Which on is the tertiary consumer of sundarbans?   
3           Which hormone is sybtgesuzed in leaves?   
4                 Which one is used as insecticide?   

                               Bengali Question              A             B  \
0                        তেঁতুলে কোন এসিড থাকে?          Malic        Citric   
1  কোন মৌল দুটির সাহায্যে ফিউজ প্রস্তুত করা হয়?  Lead and Zinc  Zinc and Tin   
2          সুন্দরবনের তৃতীয় স্তরের খাদক কোনটি?            Pig          Deer   
3                     পাতায় উৎপন্ন হরমোন কোনটি?          Auxin      Ethylene   
4            ফসলের ক্ষতিকর পেস্ট দমন করে কোনটি?       NH_{4}OH          NaOH   

              C              D          A Bn         B Bn        C Bn  \
0        Oxalic       Tartaric       

  0%|          | 0/228 [00:00<?, ?it/s]

Error w/ sample English Question                         How many satellites Saturn has
Bengali Question                                      শনির উপগ্রহ কয়টি?
A                                                                    63
B                                                                    34
C                                                                    27
D                                                                    13
A Bn                                                                 63
B Bn                                                                 34
C Bn                                                                 27
D Bn                                                                 13
Correct Answer                                                        *
predicted_option      (গ) স্নেহ\n\nজীবনে, স্নেহ একটি গুরুত্বপূর্ণ স্...
correct_option_fmt                                              ক) আমিষ
Name: 34, dtype: object
Error w/ sample English 

  0%|          | 0/396 [00:00<?, ?it/s]

                                    English Question  \
0  What is the value of \int_{0}^{1} \frac{sin^{-...   
1  The equation of a curve is y = 3x (x - 2).\n \...   
2  What is the value of \int e^{x} \left ( \frac{...   
3  The value of \int_{1}{\sqrt{3}} \frac{x}{1 + x...   
4  If f(x) = 2x then-\n i. \int \frac{dx}{f(x)} =...   

                                    Bengali Question                    A  \
0  \int_{0}^{1} \frac{sin^{-1}p}{\sqrt{1-p^{2}}}d...        \frac{\pi}{2}   
1  y = 3x (x - 2) একটি বক্ররেখার সমীকরণ।\n \n মূল...           y + 6x = 0   
2  \int e^{x} \left ( \frac{1}{x} - \frac{1}{x^{2...  - \frac{e^x}{x} + c   
3    \int_{1}{\sqrt{3}} \frac{x}{1 + x^2} dx এর মান-                  ln2   
4  f(x) = 2x হলে-\n i. \int \frac{dx}{f(x)} = \fr...             i and ii   

                       B                         C                    D  \
0      \frac{\pi^{2}}{2}         \frac{\pi^{2}}{4}    \frac{\pi^{2}}{8}   
1             y - 6x = 0                x + 6y = 0

...

                                    English Question  \
0  An arrow throws into a wall losses half its ve...   
1              What is the value of \sqrt[4]{- 49} ?   
2  If \alpha, \beta, \gamma be the roots of the e...   
3  If \frac{1}{2 + i} be the one of the roots of ...   
4  If 3 be the one of the roots of x^{2} - kx + 2...   

                                    Bengali Question                     A  \
0  একটি তীর একটি দেয়ালের ভিতর 3 ইঞ্চি ঢুকার পর তা...           \frac{1}{2}   
1                       \sqrt[4]{- 49} এর মান কোনটি?         \pm \sqrt{7}i   
2  3x^{3} - 1 = 0 সমীকরণের মূলত্রয় \alpha, \beta,...                    -1   
3  কোনো দ্বিঘাত সমীকরণের একটি মূল \frac{1}{2 + i}...  9x^{2} - 12x + 5 = 0   
4  x^{2} - kx + 2 = 0, সমীকরণের একটি মূল 3 হলে-\n...              i and ii   

                                  B                         C  \
0                       \frac{1}{3}               \frac{2}{3}   
1  \pm \sqrt{\frac{7}{2}} (1 \pm i)  \pm \frac{7}{2} ( \pm i)   

  0%|          | 0/304 [00:00<?, ?it/s]

                                    English Question  \
0  A spring is expanded 10cm by appling 500N forc...   
1  Which one is the angular velocity of the natio...   
2  The maximum velocity and maximum acceleration ...   
3  The maximum velocity and maximum acceleration ...   
4  Vector \frac{\hat{i} - \hat{j} + \hat{k}}{\sqr...   

                                    Bengali Question  \
0  একটি স্প্রিং-এ 500 N বল প্রয়োগ করায় স্প্রিংটি ...   
1  পৃথিবীর নিজ অক্ষের ঘূর্ণনের জন্য জাতীয় স্মৃতিস...   
2  সরল ছন্দিত গতিতে দোলনরত একটি কণার সর্বোচ্চ ত্ব...   
3  সরল ছন্দিত গতিতে দোলনরত একটি কণার সর্বোচ্চ ত্ব...   
4  \frac{\hat{i}-\hat{j}+\hat{k}}{\sqrt{3}} ভেক্ট...   

                              A                 B  \
0                         0.04J             1.90J   
1  7.27 \times 10^{5} rads^{-1}  0.2618 rads^{-1}   
2                        0.785s             1.57s   
3                        i & ii           i & iii   
4                   null vector  co-linear vector   

       

  0%|          | 0/333 [00:00<?, ?it/s]

                                    English Question  \
0  The ratio of amplitude of two co-herent source...   
1  The maximum magnitude of electric field of an ...   
2  A carnot engine when remain at 72\degree C sin...   
3  A carnot engine when remain at 72\degree C sin...   
4  The phase difference between two points of an ...   

                                    Bengali Question  \
0  দুটি সুসংগত উৎসের বিস্তারের অনুপাত 3 : 2 । তরঙ...   
1  একটি তড়িৎচুম্বকীয় তরঙ্গের তড়িৎ ক্ষেত্রের সর্বো...   
2  একটি কার্নো ইঞ্জিন যখন 72\degree C তাপমাত্রায় ...   
3  একটি কার্নো ইঞ্জিন যখন 72\degree C তাপমাত্রায় ...   
4  একটি তড়িৎ চুম্বকীয় তরঙ্গের উপর অবস্থিত যেকোনো ...   

                                  A                           B  \
0                             9 : 4                       4 : 9   
1  1.62 \times 10^{10}_{1} Wbm^{-2}  1.8 \times 10^{7} Wbm^{-2}   
2                             138 K                       207 K   
3                                 i                       

In [18]:
import re
import pandas as pd

# from rouge import Rouge
# from bert_score import score as bert_score
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelBinarizer

def extract_predicted_answer(text):
    """
    Extract the predicted answer from the large chunk of text by looking for
    the '### আপনার উত্তর:' section and isolating the answer.
    """
    # Try to find the part of the text after '### আপনার উত্তর:'
    match = re.search(r'### আপনার উত্তর:\s*(.*)', text)

    # If found, return the extracted answer, otherwise return the original text
    if match:
        return match.group(1).strip()  # Extract and clean the answer
    return text.strip()  # If no match, return the text as-is

def clean_text_bangla(text):
    """
    Perform basic cleaning of Bangla text.
    """
    # Replace newline characters and extra spaces
    text = text.replace("\n", " ").replace("\\n", " ").replace("\r", " ")

    # Remove unnecessary special characters while keeping Bangla characters
    text = re.sub(r'[^\u0980-\u09FF\s]', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def evaluate_rqa(df):
    # Extract the relevant predicted answer from the large predicted text
    df['cleaned_predicted_option'] = df['predicted_option'].apply(extract_predicted_answer)

    # Clean the extracted text (to remove any remaining special characters)
    df['correct_option_fmt'] = df['correct_option_fmt'].apply(clean_text_bangla)
    df['cleaned_predicted_option'] = df['cleaned_predicted_option'].apply(clean_text_bangla)

    # Proceed with evaluation
    y_true = df['correct_option_fmt']
    y_pred = df['cleaned_predicted_option']

    # Calculate accuracy, precision, recall, and f1_score as before
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    # Binarize the labels for AUC-ROC and log-loss calculations
    lb = LabelBinarizer()
    y_true_binarized = lb.fit_transform(y_true)
    y_pred_binarized = lb.transform(y_pred)

    auc_roc = roc_auc_score(y_true_binarized, y_pred_binarized, average='macro', multi_class='ovr')
    logloss = log_loss(y_true_binarized, y_pred_binarized)

    # Return the metrics
    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'log_loss': logloss
    }

    return metrics

In [20]:
import pandas as pd
file_tags = [
    "8th_science",
    "12th_math_1",
    "12th_math_2",
    "12th_physics_1",
    "12th_physics_2"
]
prompts_tags = [
    "zs"
]

for file_index in range(5):
  for prompts_index in range(1):
    eval_df = pd.read_parquet(path_root + f"llama_{prompts_tags[prompts_index]}_rqa_{file_tags[file_index]}_results.parquet")
    print(path_root + f"llama_{prompts_tags[prompts_index]}_rqa_{file_tags[file_index]}_results.parquet")

    metrics = evaluate_rqa(eval_df)
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

exps/llama_zs_rqa_8th_science_results.parquet
accuracy: 0.0965
precision: 0.0482
recall: 0.0436
f1_score: 0.0441
auc_roc: 0.5478
log_loss: 32.5658
exps/llama_zs_rqa_12th_math_1_results.parquet


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2956: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2956: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


accuracy: 0.1263
precision: 0.0162
recall: 0.0097
f1_score: 0.0115
auc_roc: 0.5505
log_loss: 31.4927
exps/llama_zs_rqa_12th_math_2_results.parquet
accuracy: 0.1074
precision: 0.0206
recall: 0.0119
f1_score: 0.0137
auc_roc: 0.5473
log_loss: 32.1720
exps/llama_zs_rqa_12th_physics_1_results.parquet
accuracy: 0.1513
precision: 0.0816
recall: 0.0745
f1_score: 0.0767
auc_roc: 0.6028
log_loss: 30.5897
exps/llama_zs_rqa_12th_physics_2_results.parquet
accuracy: 0.1051
precision: 0.0474
recall: 0.0448
f1_score: 0.0449
auc_roc: 0.5610
log_loss: 32.2553


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2956: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2956: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2956: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
